<a href="https://colab.research.google.com/github/SoumyadeepB/DeepLearning_Coding_Task/blob/master/NLU_CNN_singleAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dense, Input, Flatten, Conv2D,Conv1D, MaxPooling2D,MaxPooling1D,GlobalMaxPooling1D,GlobalMaxPooling2D
from keras.layers import Reshape, Dropout, Concatenate, LSTM,Bidirectional,BatchNormalization
from keras.layers import Flatten,Activation,CuDNNGRU,CuDNNLSTM,LeakyReLU, Concatenate,concatenate
from keras.models import Model
from keras.models import Sequential
from keras import regularizers,optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, MinMaxScaler,RobustScaler
from sklearn.metrics import accuracy_score
import os
import json
from IPython.core.display import display, HTML
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


In [0]:
PAD_ROWS=26
PAD_COLS=600

In [4]:
print(os.listdir('drive/My Drive/DL Dataset'))
dir = 'drive/My Drive/DL Dataset/'
f = open(os.path.join(dir, 'train.json'), encoding="utf8")
data = json.load(f)

['dev.json', 'train.json', 'valence_1000.h5', 'model_cnn_final.h5', 'best_model_cnn.h5', 'model_cnn.h5', 'NLU', 'res1.json']


In [5]:
df = pd.DataFrame(data)
df= df.T
TOTAL_DATA=df.shape[0]
df['labels'] = df[['valence', 'activation']].values.tolist()
df.head()

,valence,activation,features,labels
0,0,1,"[[5.502810676891276, 5.389630715979907, 5.8907...","[0, 1]"
1,1,1,"[[5.059076172970736, 5.288492317702101, 4.2633...","[1, 1]"
2,0,1,"[[4.218546271669202, 4.961436495859291, 3.6650...","[0, 1]"
3,1,0,"[[4.650364321573866, 4.523905028353254, 5.0168...","[1, 0]"
4,0,1,"[[3.900221957277269, 2.7325726489808124, 2.565...","[0, 1]"


In [6]:
### DATA AUGMENTATION  - To Use a concatenated layer

# For input to CNN : Shape: (N x 26)

label_set=pd.DataFrame(np.zeros((TOTAL_DATA,1),dtype=np.int),columns=['labels'])
code=0
scaler = MinMaxScaler(feature_range=(0.1, 1))

# Use only 600 features to learn
# ------------------------------------------------ Get Label Code ----------------------------------------------------------
for i in tqdm_notebook(range(TOTAL_DATA)):
    v = df['valence'][i]
    a = df['activation'][i]
    
    if v==0 and a==0:
        code=0
    elif v==0 and a==1:
        code=1
    elif v==1 and a==0:
        code=2
    elif v==1 and a==1:
        code=3
    label_set['labels'][i]=code



In [14]:
# ------------------------------------------------ Get Features with Augmentation ----------------------------------------------------------
feat1 = np.zeros((TOTAL_DATA,PAD_COLS, PAD_ROWS),dtype=np.float32)
feat2 = np.zeros((TOTAL_DATA,PAD_COLS, PAD_ROWS),dtype=np.float32)
scaler = MinMaxScaler(feature_range=(0.1, 1))
robust = RobustScaler()
for i in tqdm_notebook(range(TOTAL_DATA)):
    x = np.array(df['features'][i])[:PAD_COLS,:]
    [r,c] = x.shape
    # feat1[i,:r,:c] = scaler.fit_transform(np.float32(x))   # Scale to [0,1] range
    # feat2[i,:r,:c] = scaler.fit_transform(np.float32(x+1.5))
    feat1[i,:r,:c] = robust.fit_transform(np.float32(x))   # Scale to [0,1] range
    feat2[i,:r,:c] = robust.fit_transform(np.float32(x+1.5))

In [0]:
labels=label_set['labels'].astype(int)
labels = pd.get_dummies(label_set['labels']).values.tolist()
X = np.concatenate((feat1,feat2), axis=0)
Y = np.array(labels)
Y1=Y
Y=np.vstack((Y,Y1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=98)

input_dim=X[0].shape

In [16]:

Y.shape

(15600, 4)

In [0]:
ACCURACY_THRESHOLD = 0.96
class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('categorical_accuracy') > 0.90 and logs.get('val_loss') <0.2 and logs.get('val_categorical_accuracy') > ACCURACY_THRESHOLD):
			print("\nReached %2.2f%% accuracy. Stopping training!!" %(ACCURACY_THRESHOLD*100))
			self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()


In [18]:
warnings.filterwarnings("ignore")
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=9, padding='same',input_shape=input_dim,name='Conv1'))
model.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=6, padding='same',name='Conv2'))
model.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=3, padding='same',name='Conv5'))
model.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())

model.add(Dense(4, activation='softmax'))

ada = optimizers.Adadelta(decay=1e-6)
rms = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
#sgd = optimizers.SGD(lr=0.001, decay=1e-3, momentum=0.9, nesterov=True)
model.compile(optimizer=ada,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

history=model.fit(X_train, y_train, use_multiprocessing=True, verbose=1, epochs=1000, validation_data=(X_test, y_test), shuffle=True, callbacks=[callbacks])  # train the model


Train on 14040 samples, validate on 1560 samples
Epoch 1/1000
14040/14040 [==============================] - 8s 605us/step - loss: 1.5371 - categorical_accuracy: 0.3909 - val_loss: 1.4944 - val_categorical_accuracy: 0.4013
Epoch 2/1000
14040/14040 [==============================] - 7s 513us/step - loss: 1.2921 - categorical_accuracy: 0.4151 - val_loss: 1.1626 - val_categorical_accuracy: 0.4622
Epoch 3/1000
14040/14040 [==============================] - 7s 519us/step - loss: 1.2414 - categorical_accuracy: 0.4293 - val_loss: 1.1448 - val_categorical_accuracy: 0.4628
Epoch 4/1000
14040/14040 [==============================] - 7s 524us/step - loss: 1.2095 - categorical_accuracy: 0.4448 - val_loss: 1.1501 - val_categorical_accuracy: 0.4679
Epoch 5/1000
14040/14040 [==============================] - 7s 523us/step - loss: 1.1845 - categorical_accuracy: 0.4623 - val_loss: 1.2108 - val_categorical_accuracy: 0.4365
Epoch 6/1000
14040/14040 [==============================] - 7s 517us/step - loss:

KeyboardInterrupt: ignored

In [20]:
model.fit(X_train, y_train, verbose=1, initial_epoch=600, epochs=1000, validation_data=(X_test, y_test), shuffle=True, callbacks=[callbacks])  # train the model

Train on 14040 samples, validate on 1560 samples
Epoch 601/1000
14040/14040 [==============================] - 8s 546us/step - loss: 0.1468 - categorical_accuracy: 0.9496 - val_loss: 0.1670 - val_categorical_accuracy: 0.9667

Reached 96.00% accuracy. Stopping training!!


In [0]:
#Saving a model Syntax:
model.save("drive/My Drive/DL Dataset/model_cnn_best1.h5")

In [0]:
f = open(os.path.join(dir, 'dev.json'), encoding="utf8")
dev = json.load(f)

In [25]:
df1 = pd.DataFrame(dev)
df1= df1.T
TOTAL_DATA=df1.shape[0]
tst = np.zeros((TOTAL_DATA,PAD_COLS, PAD_ROWS),dtype=np.float32)
scaler = MinMaxScaler(feature_range=(0.1, 1))


for i in tqdm_notebook(range(TOTAL_DATA)):
    x = np.array(df1['features'][i])[:PAD_COLS,:]
    [r,c] = x.shape
    tst[i,:r,:c] = robust.fit_transform(x)   # Scale to [0,1] range
    
print(tst.shape)


(3342, 600, 26)


In [0]:
pred = model.predict(tst)
y_label=np.argmax(y_test, axis=1)
pred=np.argmax(pred, axis=1)

In [27]:
res=pd.DataFrame(np.zeros((TOTAL_DATA,2),dtype=np.int),columns=['valence','activation'])

## Prepare output file : Decipher predicted code to valence and activation values
for i in tqdm_notebook(range(TOTAL_DATA)):
    
    x=pred[i]
    if x==0:        
       v=0
       a=0
    elif x==1:
        v=0
        a=1
    elif x==2:
        v=1
        a=0
    elif x==3:
        v=1
        a=1
    res['valence'][i] = v
    res['activation'][i] = a

In [0]:
res.to_json('drive/My Drive/DL Dataset/NLU/zres99f.json',orient='index')

In [0]:
model.fit(X_train, y_train, verbose=1, initial_epoch=380, epochs=1000, validation_data=(X_test, y_test), shuffle=True, callbacks=[callbacks])  # train the model

In [0]:
model.summary()

In [0]:
feat_merged=np.concatenate((feat, feat1), axis=1)

In [0]:
feat_merged.shape